In [15]:
import networkx as nx
import pandas as pd
import pickle as pk
import numpy as np
from scipy import sparse as sp


import matplotlib as plt

%matplotlib inline

import seaborn as sns
plt.style.use('seaborn-whitegrid')

title_font = {'size':'18', 'color':'black', 'verticalalignment':'bottom',  'fontstyle':'bold'} 
axis_font = { 'size':'16'}

In [16]:
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_colwidth', 100)
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

authors = pd.read_csv("nips-19872020/authors.csv")
authors = authors.head(500)
institutions = pd.read_csv("nips-19872020/institutions.csv")
papers = pd.read_csv("nips-19872020/papers.csv")

In [17]:
print('The NIPS collection contains %s papers from %s authors.'%(len(authors.paper_index.unique()), len(authors.name.unique())))

The NIPS collection contains 117 papers from 497 authors.


In [18]:
int_pid = dict(enumerate(list(authors.paper_index.unique())))
int_aid = dict(enumerate(list(authors.name.unique())))

pid_intid = {v:k for k,v in int_pid.items()}
aid_intid = {v:k for k,v in int_aid.items()}

In [19]:
ap_tuples = list(zip(authors.name, authors.paper_index))
ap_int_tups =  [(aid_intid[i[0]], pid_intid[i[1]]) for i in ap_tuples]
''' AP: matrix of author-paper, AP[i, j]=1 indicates that author i has published paper j '''
AP = sp.csc_matrix((np.ones(len(ap_int_tups)), zip(*ap_int_tups)))

In [20]:
df = pd.DataFrame(ap_int_tups)

In [21]:
df['weight'] = 10
df.columns = ['Source', 'Target', 'Weight']

In [22]:
df['Source'] = df['Source'].apply(lambda x: int_aid.get(x)).fillna('')
df['Target'] = df['Target'].apply(lambda x: int_aid.get(x)).fillna('')
df

,Source,Target,Weight
0,Seongmin Ok,Seongmin Ok,10
1,Sangnie Bhardwaj,Sangnie Bhardwaj,10
2,Ian Fischer,Sangnie Bhardwaj,10
3,Johannes Ballé,Sangnie Bhardwaj,10
4,Troy Chinen,Sangnie Bhardwaj,10
...,...,...,...
495,Bertrand Charpentier,Tianju Xue,10
496,Daniel Zügner,Tianju Xue,10
497,Stephan Günnemann,Tianju Xue,10
498,Johannes Bausch,Ryan P. Adams,10


In [13]:
!pip install pyvis
from pyvis.network import Network
net = Network()
from pyvis.network import Network
import pandas as pd


In [14]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

# set the physics layout of the network
got_net.barnes_hut()
got_data = df

sources = got_data['Source']
targets = got_data['Target']
weights = got_data['Weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])

got_net.show('graph.html')